# Compute DSM step by step

This notebook explains how to run step by step DSM computation with CARS, starting from the prepare step output folder.

## Notebook parameters

Those parameters need to be set before running the notebook.

In [1]:
# Path to the cars folder
cars_home = "/work/OT/siaa/3D/Development/guinetj/cars-hal/cars/"
# Path to the directory containing the content.json file of the prepare step output
content_dir = "/work/OT/siaa/3D/Temporary/guinetj/Shareloc/Data/couple_paca_roi1/"
# ROI to process (roi_file = path to a vector file, raster file or roi_bbox=bounding box), as expected by cars_cli
# Use one or the other (roi_file will have precedence if not None)
roi_file = "/work/OT/siaa/3D/Temporary/guinetj/Shareloc/Data/couple_paca_roi1/envelopes_intersection.gpkg" # Put roi_file=None to use roi_bbox
roi_bbox = ["xmin", "ymin", "xmax", "ymax"] # Use 4 floats value
# Path to output dir where to save figures and data
output_dir = "/work/OT/siaa/3D/Temporary/guinetj/Shareloc/Data/couple_paca_roi1/out/"

import os
# path to shareloc test data
os.environ["TESTPATH"] = "/work/OT/siqi/guinetj/ShareLoc/shareloc/valid"

## Test shareloc

In [2]:
from shareloc.grid import grid
id_scene_left = "P1BP--2017092838284574CP"
id_scene_right = "P1BP--2017092838319324CP"
datum = 'ellipsoide'

data_folder = os.path.join(os.environ["TESTPATH"], datum, id_scene_left)
gld_left = os.path.join(data_folder,'grilles_gld_xH/{}_H1.hd'.format(id_scene_left))
gri_left = grid(gld_left)

data_folder = os.path.join(os.environ["TESTPATH"], datum, id_scene_right)
gld_right = os.path.join(data_folder,'grilles_gld_xH/{}_H1.hd'.format(id_scene_right))
gri_right = grid(gld_right)

grid_left_filename = os.path.join(content_dir, "left_epipolar_grid.tif")
grid_right_filename = os.path.join(content_dir, "right_epipolar_grid.tif")

FileNotFoundError: [Errno 2] No such file or directory: '/work/OT/siqi/guinetj/ShareLoc/shareloc/valid/ellipsoide/P1BP--2017092838284574CP/grilles_gld_xH/P1BP--2017092838284574CP_H1.hd'

## Imports

In [ ]:
### Trick to override cars version
import sys
sys.path = [cars_home] + sys.path

import math
os.environ['OTB_APPLICATION_PATH'] = os.path.join(cars_home,'build','lib','otb','applications')+':'+os.environ['OTB_APPLICATION_PATH']
###
# Silent OTB info logs
os.environ['OTB_LOGGER_LEVEL']='WARNING'
import warnings
warnings.filterwarnings("ignore",category=UserWarning)
import xarray as xr
import numpy as np
%matplotlib inline
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource
from matplotlib import pyplot as plt

from cars import stereo, parameters, configuration_correlator, rasterization, utils, projection, tiling, filtering
from bin.cars_cli import parse_roi_file
import pandora

## Read data

First step is to read the content file from the prepare step, and to retrieve the disparity range. Note that disparity range can be changed here for experimentations.

In [ ]:
conf = parameters.read_preprocessing_content_file(os.path.join(content_dir,'content.json'))
disp_min = int(math.floor(conf['preprocessing']['output']['minimum_disparity']))
disp_max = int(math.ceil(conf['preprocessing']['output']['maximum_disparity']))
print(disp_min)

We also need to set up a configuration for pandora, the disparity estimation algorithm.

In [ ]:
corr_config = configuration_correlator.configure_correlator()

## Compute the Region Of Interest to Process

First, we need to define the region of interest that will be processed by the notebook. For that, use `roi_file` ( a vector or a raster file) or `roi_bbox` (four float array).

In [ ]:
if roi_file is not None:
    bounds, stop_now = parse_roi_file(roi_file, stop_now=True)
else: 
    bounds = (roi_bbox, None)
print("Bounds: {}, EPSG={}".format(bounds[0], bounds[1]))

Now, we need to compute the corresponding epipolar region:

In [ ]:
epipolar_region = stereo.transform_terrain_region_to_epipolar(bounds[0], conf, bounds[1], disp_min, disp_max)
print("Corresponding epipolar region: {} (size: {} x {} pixels)".format(epipolar_region, epipolar_region[2]-epipolar_region[0], epipolar_region[3]-epipolar_region[1]))

## Stereo-rectify images

Before rectifying the images, we need to compute the margins needed by the disparity estimation algorithm:

In [ ]:
margins = pandora.marge.get_margins(disp_min, disp_max, corr_config)
margins

Now we can call the images rectification function. It will return 3 datasets, respectively for the left image, right image and left color image.

In [ ]:
left_dataset, right_dataset, left_color_dataset = stereo.epipolar_rectify_images(conf, epipolar_region, margins)

Lets display the left and right images with their masks, and see if epipolar geometry is ok with an horizontal red line.

In [ ]:
fig_size = 15
clip_percent = 5
vmin_left = np.percentile(left_dataset.im,clip_percent)
vmax_left = np.percentile(left_dataset.im,100-clip_percent)
vmin_right = np.percentile(right_dataset.im,clip_percent)
vmax_right = np.percentile(right_dataset.im,100-clip_percent)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2), subplot_kw={'aspect': 1})
axes[0].set_title("Left image")
axes[0].imshow(left_dataset.im, cmap="gray", interpolation='spline36', vmin=vmin_left, vmax=vmax_left)
axes[0].imshow(left_dataset.msk.where(left_dataset.msk !=0), cmap='tab10', alpha=0.5)
axes[0].axhline(len(left_dataset.row)/2., color='red')
axes[1].set_title("Right image")
axes[1].imshow(right_dataset["im"], cmap="gray", interpolation='spline36', vmin=vmin_right, vmax=vmax_right)
axes[1].imshow(right_dataset.msk.where(right_dataset.msk !=0), cmap='tab10', alpha=0.5)
axes[1].axhline(len(right_dataset.row)/2., color='red')
fig.tight_layout()
fig.savefig(os.path.join(output_dir,'epipolar_images.pdf'))

## Compute Disparity

Now that we have stereo-rectified images, we can compute the disparity:

In [ ]:
disp = stereo.compute_disparity(left_dataset, right_dataset, corr_config, disp_min, disp_max, verbose = True, use_sec_disp=False)

We crop out margins, since we do not need them anymore.

In [ ]:
left_dataset, tmp = xr.align(left_dataset,disp['ref'])

## Triangulation

Now that we estimated the disparity, we can triangulate it to get WGS84 3D points:

In [ ]:
import time
now = time.time()
cloud = stereo.triangulate(conf, disp['ref'])
elapsed = time.time() - now
print("elapsed time {}".format(elapsed))
cloud['ref'].to_netcdf(os.path.join(output_dir, "cloud_WGS84.nc"))
cloud['ref'] = projection.points_cloud_conversion_dataset(cloud['ref'], 4978)
cloud['ref'].to_netcdf(os.path.join(output_dir, "cloud_ECEF.nc"))  

## Triangulation shareloc

In [ ]:
from shareloc.triangulation.triangulation import epipolar_triangulation


now = time.time()
point_ecef, point_wgs84, residuals = epipolar_triangulation(disp['ref'], None, 'disp', gri_left, gri_right, grid_left_filename,
                                                   grid_right_filename, residues = True)
elapsed = time.time() - now
print("elapsed time {}".format(elapsed))

## Comparison

In [ ]:
array_shape = disp['ref'].disp.values.shape
array_epi_ecef = point_ecef.reshape((array_shape[0], array_shape[1], 3))


ecef_x = cloud['ref'].x.values
ecef_y = cloud['ref'].y.values
ecef_z = cloud['ref'].z.values
diff_x = ecef_x - array_epi_ecef[:,:,0]
diff_y = ecef_y - array_epi_ecef[:, :, 1]
diff_z = ecef_z - array_epi_ecef[:, :, 2]

mean_x = np.mean(diff_x)
min_x = np.min(diff_x)
max_x = np.max(diff_x)
print(" diff x {} {} {}".format(mean_x,min_x,max_x))
mean_y = np.mean(diff_y)
min_y = np.min(diff_y)
max_y = np.max(diff_y)
print(" diff y {} {} {}".format(mean_y,min_y,max_y))
mean_z = np.mean(diff_z)
min_z= np.min(diff_z)
max_z = np.max(diff_z)
print(" diff z {} {} {}".format(mean_z,min_z,max_z)) 

## Saving data

Saving left, right and left color datasets:

In [ ]:
left_dataset.to_netcdf(os.path.join(output_dir, "left_dataset.nc"))
right_dataset.to_netcdf(os.path.join(output_dir, "right_dataset.nc"))
left_color_dataset.to_netcdf(os.path.join(output_dir, "left_color_dataset.nc"))

Saving disparity:

In [ ]:
disp['ref'].to_netcdf(os.path.join(output_dir, "disparity.nc"))

Saving Shareloc augmented disparity:

In [ ]:
def create_dataset(disp,point_wgs84,point_ecef,residuals):
    array_shape = disp.disp.values.shape
    array_epi_wgs84 = point_wgs84.reshape((array_shape[0], array_shape[1], 3))
    array_epi_ecef = point_ecef.reshape((array_shape[0], array_shape[1], 3))
    array_residuals = residuals.reshape((array_shape[0], array_shape[1]))
    pc_dataset = xr.Dataset({'pc_wgs84_x': (['row', 'col'], array_epi_wgs84[:, :, 0]),
                             'pc_wgs84_y': (['row', 'col'], array_epi_wgs84[:, :, 1]),
                             'pc_wgs84_z': (['row', 'col'], array_epi_wgs84[:, :, 2]),
                             'pc_ecef_x': (['row', 'col'], array_epi_ecef[:, :, 0]),
                             'pc_ecef_y': (['row', 'col'], array_epi_ecef[:, :, 1]),
                             'pc_ecef_z': (['row', 'col'], array_epi_ecef[:, :, 2]),
                             'residues': (['row', 'col'], array_residuals)},
                            coords={'row': disp.coords['row'], 'col': disp.coords['col'] })
    return pc_dataset

pc_dataset = create_dataset(disp['ref'], point_wgs84, point_ecef, residuals)
disp = xr.merge((disp['ref'], pc_dataset))
out_disp_filename = os.path.join(output_dir, "out_disparity_shareloc_grid.nc")
disp.to_netcdf(out_disp_filename)

Saving triangulated cloud:

In [ ]:
cloud['ref'].to_netcdf(os.path.join(output_dir, "cloud.nc"))

Saving DSM:

In [ ]:
raster.to_netcdf(os.path.join(output_dir, "dsm.nc"))

Save cloud in ply file format:

In [ ]:
utils.write_ply(os.path.join(output_dir,"points.ply"),cloud['ref'])
if small_cpn_filter_params is not None or statistical_filter_params is not None:
    utils.write_ply(os.path.join(output_dir,"filtered_points.ply"),filtered_points)